# Feature Engineering (Ingenieria de Atributos)

Lucho Nov| 2025 <br>
Coconut by Harry Nilsson <br>

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from ta.momentum import RSIIndicator, StochasticOscillator, ROCIndicator, WilliamsRIndicator
from ta.trend import SMAIndicator, EMAIndicator, MACD, ADXIndicator, CCIIndicator
from ta.volatility import AverageTrueRange, BollingerBands

In [2]:
file_path = r"E:\data_lightstorm\data\raw\EURGBP_2020_2024.csv"

try:
    # Leer CSV, eliminar duplicados y rellenar nulos hacia adelante
    df = (
        pl.read_csv(file_path)
        .unique()
        .fill_null(strategy="forward")
    )

    df_raw = df.clone()

    # Convertir columna 'date' a datetime (sin fmt=)
    df = df.with_columns(
        pl.col("date").str.strptime(pl.Datetime, strict=False).alias("date")
    )

    # Verificar fechas inválidas
    invalid_dates = df["date"].null_count()
    if invalid_dates > 0:
        print(f"⚠️ {invalid_dates} filas con fecha inválida eliminadas.")
        df = df.filter(pl.col("date").is_not_null())

    # Mostrar primeras filas
    print(df.head())

except FileNotFoundError:
    print(f"❌ Archivo no encontrado: {file_path}")
except Exception as e:
    print(f"⚠️ Error al cargar el archivo: {e}")

shape: (5, 5)
┌─────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ date                ┆ open    ┆ high    ┆ low     ┆ close   │
│ ---                 ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ datetime[μs]        ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞═════════════════════╪═════════╪═════════╪═════════╪═════════╡
│ 2023-11-08 00:00:00 ┆ 0.86985 ┆ 0.87177 ┆ 0.8691  ┆ 0.87169 │
│ 2020-05-26 00:00:00 ┆ 0.89373 ┆ 0.89414 ┆ 0.88799 ┆ 0.8903  │
│ 2023-05-04 00:00:00 ┆ 0.88009 ┆ 0.88136 ┆ 0.87453 ┆ 0.87588 │
│ 2024-03-19 00:00:00 ┆ 0.85405 ┆ 0.85557 ┆ 0.85319 ┆ 0.85395 │
│ 2020-01-01 00:00:00 ┆ 0.84548 ┆ 0.84832 ┆ 0.84505 ┆ 0.84586 │
└─────────────────────┴─────────┴─────────┴─────────┴─────────┘


## Feature Engineering

📊 Variables Técnicas (Indicadores Financieros)

- **SMA_30**: Media móvil simple de 30 días del precio de cierre.  
- **SMA_90**: Media móvil simple de 90 días del precio de cierre.  
- **SMA_crossover**: Diferencia entre `SMA_30` y `SMA_90`.  
- **RSI**: Índice de Fuerza Relativa (14 períodos).  
- **ATR**: Rango Verdadero Promedio (14 períodos).  
- **volatility_30d**: Desviación estándar de los retornos diarios (30 días).  

---

📅 Variables Temporales

- **month**: Mes del año (1–12).  
- **quarter**: Trimestre del año (1–4).  
- **day_of_week**: Día de la semana (0–6).  
- **is_month_end**: Booleano para fin de mes.  

---

🔁 Variables Relacionales / Derivadas

- **sma_ratio**: `SMA_30 / SMA_90`.  
- **return_t**: Retorno porcentual del día *t*:  
  \[
  (close_t / close_{t-1}) - 1
  \]
- **volatility_rolling**: Volatilidad móvil (10 días).  

---

📈 Variables de Volumen (Agregadas por relevancia)

- **volume_ratio**: Volumen del día / Media móvil de volumen (30 días).  
- **volume_trend**: Tendencia de volumen (pendiente de regresión lineal 5 días).  

---

🎯 TARGET

**target**: Clasificación ternaria basada en `return_{t+1}`:

- 🟩 **up** si `return_{t+1} > 0.001`  
- 🟥 **down** si `return_{t+1} < -0.001`  
- ⬜ **neutral** en otro caso  

In [3]:
class ForexFeatureEngineer:
    """
    Clase para engineering de features para datos Forex EURGBP
    """
    
    def __init__(self):
        self.feature_columns = []
        
    def create_technical_features(self, df):
        """
        Crear features técnicas usando la librería ta
        """
        print("Creando features técnicas...")
        
        # Asegurar que tenemos las columnas necesarias
        required_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Columna requerida '{col}' no encontrada en el DataFrame")
        
        # 1. INDICADORES DE TENDENCIA (SMA)
        df['SMA_30'] = ta.trend.sma_indicator(df['close'], window=30)
        df['SMA_90'] = ta.trend.sma_indicator(df['close'], window=90)
        df['SMA_crossover'] = df['SMA_30'] - df['SMA_90']
        df['sma_ratio'] = df['SMA_30'] / df['SMA_90']
        
        # 2. INDICADORES DE MOMENTUM (RSI)
        df['RSI'] = ta.momentum.rsi(df['close'], window=14)
        
        # 3. INDICADORES DE VOLATILIDAD (ATR)
        df['ATR'] = ta.volatility.average_true_range(
            df['high'], df['low'], df['close'], window=14
        )
        
        # 4. VOLATILIDAD HISTÓRICA
        df['returns'] = df['close'].pct_change()
        df['volatility_30d'] = df['returns'].rolling(window=30).std()
        df['volatility_rolling'] = df['returns'].rolling(window=10).std()
        
        # 5. FEATURES DE VOLUMEN
        df['volume_ratio'] = df['volume'] / df['volume'].rolling(window=30).mean()
        
        # Tendencia de volumen (pendiente de regresión lineal 5 días)
        def calculate_volume_trend(volume_series):
            if len(volume_series) < 5 or volume_series.isna().any():
                return np.nan
            x = np.arange(len(volume_series))
            slope = np.polyfit(x, volume_series, 1)[0]
            return slope
        
        df['volume_trend'] = df['volume'].rolling(window=5).apply(
            calculate_volume_trend, raw=False
        )
        
        # 6. RETORNO DEL DÍA ACTUAL
        df['return_t'] = df['close'].pct_change()
        
        # Limpiar columnas temporales
        df = df.drop(['returns'], axis=1)
        
        return df
    
    def create_temporal_features(self, df, date_column=None):
        """
        Crear features temporales a partir de la columna de fecha
        """
        print("Creando features temporales...")
        
        # Identificar columna de fecha
        if date_column and date_column in df.columns:
            df['date'] = pd.to_datetime(df[date_column])
        elif 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        else:
            # Asumir que el índice es datetime
            df = df.reset_index()
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
            else:
                raise ValueError("No se encontró columna de fecha en el DataFrame")
        
        # Extraer componentes temporales
        df['month'] = df['date'].dt.month
        df['quarter'] = df['date'].dt.quarter
        df['day_of_week'] = df['date'].dt.dayofweek  # 0=Lunes, 6=Domingo
        df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
        
        # Features cíclicas para mes y día de la semana
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        
        return df
    
    def create_target(self, df):
        """
        Crear target de clasificación ternaria
        """
        print("Creando variable target...")
        
        # Calcular retorno del día siguiente
        df['return_t1'] = df['close'].shift(-1) / df['close'] - 1
        
        # Definir condiciones para clasificación ternaria
        conditions = [
            df['return_t1'] > 0.001,    # UP: sube más del 0.1%
            df['return_t1'] < -0.001    # DOWN: baja más del 0.1%
        ]
        choices = ['up', 'down']
        
        # Aplicar condiciones
        df['target'] = np.select(conditions, choices, default='neutral')
        
        # Mapear a valores numéricos para el modelo
        target_map = {'down': 0, 'neutral': 1, 'up': 2}
        df['target_encoded'] = df['target'].map(target_map)
        
        return df
    
    def get_feature_columns(self):
        """
        Retorna la lista de columnas de features
        """
        technical_features = [
            'SMA_30', 'SMA_90', 'SMA_crossover', 'RSI', 'ATR', 
            'volatility_30d', 'volatility_rolling', 'sma_ratio', 'return_t',
            'volume_ratio', 'volume_trend'
        ]
        
        temporal_features = [
            'month', 'quarter', 'day_of_week', 'is_month_end',
            'month_sin', 'month_cos', 'day_sin', 'day_cos'
        ]
        
        return technical_features + temporal_features
    
    def prepare_features(self, df, date_column=None):
        """
        Pipeline completo de feature engineering
        """
        print("Iniciando pipeline de feature engineering...")
        
        # Hacer copia para no modificar el original
        df_processed = df.copy()
        
        # Aplicar transformaciones
        df_processed = self.create_technical_features(df_processed)
        df_processed = self.create_temporal_features(df_processed, date_column)
        df_processed = self.create_target(df_processed)
        
        # Obtener lista de columnas de features
        self.feature_columns = self.get_feature_columns()
        
        # Limpiar valores infinitos y NaN
        df_processed = df_processed.replace([np.inf, -np.inf], np.nan)
        
        # Eliminar filas con NaN en features o target
        initial_shape = df_processed.shape[0]
        df_processed = df_processed.dropna(subset=self.feature_columns + ['target_encoded'])
        final_shape = df_processed.shape[0]
        
        print(f"Filas eliminadas por NaN: {initial_shape - final_shape}")
        print(f"Filas finales: {final_shape}")
        print(f"Total features creadas: {len(self.feature_columns)}")
        
        # Distribución del target
        target_dist = df_processed['target'].value_counts()
        print("\nDistribución del target:")
        for label, count in target_dist.items():
            print(f"  {label}: {count} ({count/len(df_processed)*100:.1f}%)")
        
        return df_processed